掛接雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary packages

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import json
import math
import csv
from pathlib import Path

In [ ]:
# Define model structures and functions

class Net(nn.Module):
    def __init__(self, load_pretrained: bool = False, pretrained_model_path :str  = "None"):
        super(Net, self).__init__()
        # Define a fully connected layers model with three inputs (frequency, flux density, duty ratio) and one output (power loss).
        self.layers = nn.Sequential(
            nn.Linear(1026, 65),
            nn.ReLU(),
            nn.Linear(65, 55),
            nn.ReLU(),
            nn.Linear(55, 116),
            nn.ReLU(),
            nn.Linear(116, 40),
            nn.ReLU(),
            nn.Linear(40, 123),
            nn.ReLU(),
            nn.Linear(123, 1),
        )
        if load_pretrained and pretrained_model_path is not None:
          self.load_pretrained_model(pretrained_model_path)

    def forward(self, x):
        return self.layers(x)

    def load_pretrained_model(self, path):
      pretrained_dict = torch.load(path)
      model_dict = self.state_dict()
      model_dict.update(pretrained_dict)
      self.load_state_dict(model_dict)
      print('Model is load')


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# Load the dataset
material_name = 'Material D'

# material_b_mix   3E6_77_78_N30

B_file_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Database/{material_name}/B_Field.csv'
# B_file_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Database/{material_name}/B_waveform.csv'
Freq_file_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Database/{material_name}/Frequency.csv'
Temp_file_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Database/{material_name}/Temperature.csv'
Power_file_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Database/{material_name}/Volumetric_Loss.csv'

# Pre-Train
pretrain_model_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Final_test/FNN/Model/Test_Split/spilt622/Model_N49_optuna2_NoPretrain.sd'
Transfer_use = True
Pretrain = 'Pretrain_N49' #Pretrain/NoPretrain
spilt_way = 'spilt622' #spilt622/spilt2/midterm_material

# Output
output_sd_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Final_test/FNN/Model/Test_Split/{spilt_way}/Model_{material_name}_optuna2_{Pretrain}.sd'
output_pred_path = f'/content/drive/MyDrive/Colab_Notebooks/MagNet/Final_test/FNN/Loss/withoutH_Transfer/{spilt_way}/pred_loss_{material_name}_optuna2_{Pretrain}.csv'

def get_dataset():

    B = read_csv(B_file_path)
    Freq = read_csv(Freq_file_path)
    Temp = read_csv(Temp_file_path)
    #H = read_csv(H_file_path)
    Power = read_csv(Power_file_path)

    # Compute labels
    # There's approximalely an exponential relationship between Loss-Freq and Loss-Flux.
    # Using logarithm may help to improve the training.
    Freq = np.log10(Freq)
    Temp = np.array(Temp)
    Power = np.log10(Power)

    # Reshape data
    Freq = torch.from_numpy(Freq).float().view(-1, 1)
    B = torch.from_numpy(B).float().view((-1,1024,1))
    #H = torch.from_numpy(H).float().view((-1,1024,1))
    Temp = torch.from_numpy(Temp).view(-1, 1)
    Power = Power.reshape((-1,1))

    # Normalize
    B = (B-torch.mean(B))/torch.std(B).numpy()
    #H = (H-torch.mean(H))/torch.std(H).numpy()
    Freq = (Freq-torch.mean(Freq))/torch.std(Freq).numpy()
    Temp = (Temp-torch.mean(Temp))/torch.std(Temp).numpy()

    B = np.squeeze(B, axis=2)
    #H = np.squeeze(H, axis=2)

    print(np.shape(Freq))
    print(np.shape(B))
    #print(np.shape(H))
    print(np.shape(Temp))
    print(np.shape(Power))

    temp = np.concatenate((Freq,B,Temp),axis=1)

    in_tensors = torch.from_numpy(temp).view(-1, 1026)
    out_tensors = torch.from_numpy(Power).view(-1, 1)

    return torch.utils.data.TensorDataset(in_tensors, out_tensors)

def read_csv(file_path):
    data = []
    with open(file_path, 'r', newline='') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            values = [float(value) for value in row]
            data.append(values)
    return np.array(data)

In [ ]:
# Config the model training

def main():
    # Reproducibility
    random.seed(1)
    np.random.seed(1)
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Hyperparameters
    NUM_EPOCH = 2000
    BATCH_SIZE = 128
    DECAY_EPOCH = 423
    DECAY_RATIO = 0.458
    LR_INI = 0.005
    best_loss = math.inf
    early_stop_count = 0
    early_stop = 500
    # Select GPU as default device
    device = torch.device("cuda")

    # Load dataset
    dataset = get_dataset()

    # Split the dataset
    train_size = int(0.6 * len(dataset))
    valid_size = int(0.2 * len(dataset))
    test_size = len(dataset) - train_size - valid_size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
    kwargs = {'num_workers': 0, 'pin_memory': True, 'pin_memory_device': "cuda"}
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, **kwargs)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, **kwargs)

    # Setup network
    # net = Net().double().to(device)
    net = Net(Transfer_use ,pretrain_model_path).double().to(device)

    # Log the number of parameters
    print("Number of parameters: ", count_parameters(net))

    # Setup optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=LR_INI)

    # Train the network
    for epoch_i in range(NUM_EPOCH):

        # Train for one epoch
        epoch_train_loss = 0
        net.train()
        optimizer.param_groups[0]['lr'] = LR_INI* (DECAY_RATIO ** (0+ epoch_i // DECAY_EPOCH))

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()

        # Compute Validation Loss
        with torch.no_grad():
            epoch_valid_loss = 0
            for inputs, labels in valid_loader:
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                epoch_valid_loss += loss.item()

        if (epoch_i+1)%5 == 0:
          print(f"Epoch {epoch_i+1:2d} "
              f"Train {epoch_train_loss / len(train_dataset) * 1e5:.5f} "
              f"Valid {epoch_valid_loss / len(valid_dataset) * 1e5:.5f}")

        # Early stop
        epoch_valid_loss = epoch_valid_loss / len(valid_dataset) * 1e5
        if epoch_valid_loss < best_loss:
          best_loss = epoch_valid_loss
          torch.save(net.state_dict(), output_sd_path)  # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
        else:
          early_stop_count += 1

        if early_stop_count >= early_stop:
          print('Model is not improving, so we halt the training session.')
          break
    print("Training finished! Model is saved!")

    # Load the best model  ====================================================
    net.load_state_dict(torch.load(output_sd_path))
    print("Best model is load to test")
    # =====================================================================

    # Evaluation
    net.eval()
    y_meas = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            y_pred.append(net(inputs.to(device)))
            y_meas.append(labels.to(device))

    y_meas = torch.cat(y_meas, dim=0)
    y_pred = torch.cat(y_pred, dim=0)
    print(f"Test Loss: {F.mse_loss(y_meas, y_pred).item() / len(test_dataset) * 1e5:.5f}")

    yy_pred = 10**(y_pred.cpu().numpy())
    yy_meas = 10**(y_meas.cpu().numpy())
    print("yy_pred :", yy_pred.shape)
    print("yy_meas :", yy_meas.shape)

    with open(output_pred_path, "w") as f:
        np.savetxt(f, (yy_pred))
        f.close()
    #with open("/content/drive/MyDrive/材料/Material B/Pred_loss/meas_loss_Material B_optuna3.csv", "w") as f:
        #np.savetxt(f, (yy_meas))
        #f.close()

    # Relative Error
    Error_re = abs(yy_pred-yy_meas)/abs(yy_meas)*100
    Error_re_avg = np.mean(Error_re)
    Error_re_rms = np.sqrt(np.mean(Error_re ** 2))
    Error_re_95prct = np.percentile(Error_re, 95)
    Error_re_99prct = np.percentile(Error_re, 99)
    Error_re_max = np.max(Error_re)

    print(f"Relative Error: {Error_re_avg:.8f}")
    print(f"AVG Error: {Error_re_avg:.8f}")
    # print(f"RMS Error: {Error_re_rms:.8f}")
    print(f"95-PRCT Error: {Error_re_95prct:.8f}")
    print(f"99th Percentile Error: {Error_re_99prct:.8f}")
    print(f"MAX Error: {Error_re_max:.8f}")


if __name__ == "__main__":
    main()

torch.Size([580, 1])
torch.Size([580, 1024])
torch.Size([580, 1])
(580, 1)
Model is load
Number of parameters:  86728
Saving model with loss 129.094...
Saving model with loss 22.465...
Epoch  5 Train 49.25980 Valid 20.60413
Saving model with loss 20.604...
Saving model with loss 20.586...
Epoch 10 Train 15.40401 Valid 12.19422
Saving model with loss 12.194...
Saving model with loss 9.411...
Epoch 15 Train 6.84011 Valid 9.44941
Saving model with loss 7.844...
Saving model with loss 6.558...
Saving model with loss 5.932...
Epoch 20 Train 3.95559 Valid 5.81277
Saving model with loss 5.813...
Saving model with loss 5.577...
Saving model with loss 4.702...
Saving model with loss 4.170...
Epoch 25 Train 2.51129 Valid 4.10282
Saving model with loss 4.103...
Saving model with loss 4.069...
Saving model with loss 3.790...
Epoch 30 Train 1.74797 Valid 4.19225
Saving model with loss 3.309...
Epoch 35 Train 1.64718 Valid 3.43012
Epoch 40 Train 1.32393 Valid 3.30126
Saving model with loss 3.301...
